In [1]:
#put unitary representations of the symmetric group over GF(q**2) together to form unitary DFT
#each rep'n is unitary, use normalizing factors \sqrt{d_\rho/|G|}
#DFT.DFT^* = S, a diagonal matrix with signs \pm 1 on diagonal. factor them as c = zz^*, so S=RR^*
#then uDFT = R^{-1}.DFT is unitary

In [23]:
def invariant_symmetric_bilinear_matrix(partition):
    """
    Computes the matrix of a S_n-invariant symmetric bilinear form.
    Sets up and solves system of linear equations based on writing U as an unknown in polynomial ring generators. 
    The equations are \rho(g)^T*U*\overline{\rho(g)} = \lambda_g*U where \lambda_g = \det(\rho(g))\overline{\det(\rho(g))}.
    The variables for U can be extracted to yield a matrix over GF(q^2) for each g.
    These are stacked to get the overall system, and we find the one dim'l null space to get a solution vector, and format as a matrix.
    Note: one could also form the Kroenecker products \rho(g) \otimes \rho(g)^{-1 T} to explicitly obtain the system.
    """
    # Define the group G and its rep'n as a Specht module, dimension
    specht_module = SGA.specht_module(partition)
    rho = specht_module.representation_matrix
    d_rho = specht_module.dimension()
    # Initialize U as a matrix of variables over GF(q^2)
    R = PolynomialRing(F, 'u', d_rho**2)
    U_vars = R.gens()  # List of variable generators for U
    U = matrix(R, d_rho, d_rho, U_vars)  # U is a d_rho x d_rho matrix of variables
    # for each generator of G, form the augmented system 
    def augmented_matrix(g):
        #form the matrix equation \rho(g)^T*U*\overline{\rho(g)} = \lambda_g * U
        #lambda_g = rho_g.det()*rho_g.det()**q
        #note: \lambda_g isn't necessary. probably only relates to sesquilinear forms
        rho_g = rho(g)
        equation_matrix = rho_g.transpose()*U*rho_g.H.transpose() - U
        # Initialize a list to hold rows of the augmented system
        augmented_system = []
        # Extract coefficients for each linear equation in the matrix
        for i in range(d_rho):
            for j in range(d_rho):
                # Get the (i, j) entry of the equation matrix, which is a linear combination of the u variables
                linear_expression = equation_matrix[i, j]
                # Extract the coefficients of each u_k in the linear expression
                row = [linear_expression.coefficient(u) for u in U_vars]
                # Append the row to the augmented system
                augmented_system.append(row)
        # Convert the augmented system to a matrix
        return matrix(F, augmented_system)
    #stack linear systems for each g in G
    total_system = matrix(F,0,d_rho**2)
    for g in G:
        total_system = total_system.stack(augmented_matrix(g))
    #compute the null space of the overall matrix
    null_space = total_system.right_kernel()
    #return a d_rho x d_rho matrix over GF(q^2) from the 1 dim'l null space given as vector
    U_mats = [matrix(F,d_rho,d_rho,b) for b in null_space.basis()]
    return U_mats

In [4]:
#for u in GF(q), we can factor as u=aa^* using gen. z and modular arithmetic
def conj_square_root(u):
    if u == 0:
        return 0  # Special case for 0
    z = F.multiplicative_generator()
    k = u.log(z)  # Compute discrete log of u to the base z
    if k % (q+1) != 0:
        raise ValueError("Unable to factor: u is not in base field GF(q)")
    return z**((k//(q+1))%(q-1))

In [5]:
#user the upper part of the LU decomposition, factor the diagonal
def base_change_hermitian(U):
    Up = U.LU()[2]
    D = Up.diagonal()
    A = ~Up * matrix.diagonal([d.sqrt() for d in D])
    diag = (A.H * U * A).diagonal()
    factor_diag = diagonal_matrix([conj_square_root(d) for d in diag])
    return factor_diag*A.inverse()

In [18]:
#define the Fourier coefficient at the rep'n specht_module corresponding to partition
def hat(f,partition,unitary=False):
    specht_module = SGA.specht_module(partition)
    rho = specht_module.representation_matrix
    if unitary:
        U = invariant_symmetric_bilinear_matrix(partition)[0]
        A = base_change_hermitian(U)
        sqrt_unitary_factor = sqrt(F(specht_module.dimension()/G.cardinality()))
        return sqrt_unitary_factor*sum(f(g)*A*rho(g)*A.inverse() for g in G)
    else:
        return sum(f(g)*rho(g) for g in G)

In [7]:
#for each basis element g \in G compute the Fourier coefficients \hat{\delta_g}(partition) for all partitions
from sage.misc.flatten import flatten
delta = lambda s: lambda t: 1 if t == s else 0 #delta function \delta_s(t)
def dft(unitary=False):
    fourier_transform = [flatten([hat(delta(g),partition,unitary).list() for partition in Partitions(G.degree())]) for g in G]
    if unitary:
        dft_matrix = matrix(F,fourier_transform).transpose()
        sign_diag = (dft_matrix*dft_matrix.H).diagonal()
        factor_diag = diagonal_matrix([conj_square_root(d) for d in sign_diag])
        return factor_diag.inverse()*dft_matrix
    else:
        return matrix(fourier_transform).transpose()

In [34]:
#parameters and define the symmetric group algebra
n = 4; q = 5
SGA = SymmetricGroupAlgebra(GF(q**2), n)
F = SGA.base_ring()
G = SGA.group()
assert F.is_finite()
assert F.order().is_square()
assert not F.characteristic().divides(G.cardinality())

In [35]:
#compute the unitary DFT
unitary_dft = dft(unitary=True); print(unitary_dft)

[       2        2        2        2        2        2        2        2        2        2        2        2        2        2        2        2        2        2        2        2        2        2        2        2]
[      z2       z2     3*z2     3*z2     3*z2     3*z2     4*z2     4*z2     2*z2     2*z2     2*z2     2*z2     2*z2     2*z2     3*z2     3*z2        0        0     2*z2     2*z2     3*z2     3*z2        0        0]
[       0        0   z2 + 3 2*z2 + 1   z2 + 3 2*z2 + 1        0        0 4*z2 + 2 3*z2 + 4 4*z2 + 2 3*z2 + 4   z2 + 3 2*z2 + 1 4*z2 + 2 3*z2 + 4   z2 + 3 4*z2 + 2   z2 + 3 2*z2 + 1 4*z2 + 2 3*z2 + 4   z2 + 3 4*z2 + 2]
[       0        0        0     2*z2        0     2*z2        0        0        0     3*z2        0     3*z2        0     2*z2        0     3*z2     2*z2     3*z2        0     2*z2        0     3*z2     2*z2     3*z2]
[       0        0        2        2        4        4        0        0        2        2        4        4        3        3  

In [36]:
#verify the resulting DFT is unitary
unitary_dft*unitary_dft.H == identity_matrix(SGA.group().cardinality())

True

In [37]:
invariant_symmetric_bilinear_matrix(partition=[3,1])

[
[1 3 3]
[3 1 3]
[3 3 1]
]